# Notebook for Serving The Trained Model for Classifying TinyImageNet. Notebook (6/6) in the End-to-End Scalable Deep Learning Pipeline on Hops.


This notebook will read the exported model that was written to HopsFS by notebook number 4 ([Notebook number two](./Step4_Ring_AllReduce_GPU_Training.ipynb)) and set it up for serving and also perform some sample queries against it.

Specifically, this notebook reads the model from:

- hdfs:///Projects/ImageNet_EndToEnd_MLPipeline/tiny-imagenet/tiny-imagenet-200/exported_model

![step5.png](./../images/step5.png)



## Create the TensorFlow Serving Through The HopsWorks UI

![serving1.png](./../images/serving_1.png)
![serving2.png](./../images/serving_2.png)
![serving3.png](./../images/serving_3.png)
![serving4.png](./../images/serving_4.png)
![serving5.png](./../images/serving_5.png)

## Test the Model By Writing a gRPC Client 

We can use the validation set and compare predictions against the labels to see where the model gets it wrong and where it gets it right.

You can inspect the exported model using tensorflow command line tool to know what inputs the model expects and what output it will give.

```bash
saved_model_cli show --dir /home/kim/workspace/imagenet_serving/1 --all
```

In future, Tensorflow will support REST clients and servers for models, but right now the framwork relies on gRPC for communication between client and server.

In gRPC a client application can directly call methods on a server application on a different machine as if it was a local object, making it easier for you to create distributed applications and services. As in many RPC systems, gRPC is based around the idea of defining a service, specifying the methods that can be called remotely with their parameters and return types.

```python
import argparse
import time
import numpy as np
from grpc.beta import implementations
from tensorflow.contrib.util import make_tensor_proto
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import scipy.ndimage
%matplotlib inline


def create_dataset(files):
    """
     A function for creating a TF dataset from TFR files
    """
    # Parse train dataset from a list of TFRecords files
    dataset = tf.data.TFRecordDataset(files,
        compression_type=None,    
        buffer_size=100240, 
        num_parallel_reads=os.cpu_count() # Parallel read from HopsFS
    )
    return dataset

def parse_tfr(example_proto):
    """
    Parses an example protocol buffer (TFRecord) into a dict of
    feature names and tensors
    """
    features = {
        'height': tf.FixedLenFeature((), tf.int64, default_value=0),
        'width': tf.FixedLenFeature((), tf.int64, default_value=0),
        'channel': tf.FixedLenFeature((), tf.int64, default_value=0),
        'label': tf.FixedLenFeature((), tf.int64, default_value=0),
        'label_one_hot': tf.FixedLenFeature((), tf.string, default_value=""),
        'image_raw': tf.FixedLenFeature((), tf.string, default_value="")
    }
    parsed_features = tf.parse_single_example(example_proto, features)
    return parsed_features["image_raw"], parsed_features["label_one_hot"]

def decode_bytes(image, label):
    """
    Decode the bytes that was serialized in the TFRecords in HopsFS to tensors so that we can apply 
    image preprocessing
    """
    image_tensor = tf.decode_raw(image, tf.uint8),
    label_tensor = tf.decode_raw(label, tf.uint8),
    image_tensor = tf.reshape(image_tensor, [64,64,3]) #dimension information was lost when serializing to disk
    return image_tensor,label_tensor

def iterate_over_dataset(dataset):
    """
    Creating an iterator over the dataset 
    """
    iterator = dataset.make_one_shot_iterator()
    next_element_op = iterator.get_next()
    return next_element_op

def get_one_batch():
    dataset = create_dataset("hdfs:///Projects/ImageNet_EndToEnd_MLPipeline/tiny-imagenet/tiny-imagenet-200/tfrecords_clean/val.tfrecords")
    dataset = dataset.map(parse_tfr)
    dataset = dataset.map(decode_bytes)
    dataset_iterate_op = iterate_over_dataset(dataset)
    sample_images = []
    sample_labels= []
    with tf.Session() as sess:
        for i in range(100):
            element = sess.run(dataset_iterate_op)
            image = element[0]
            label = element[1]
            sample_images.append(image)
            sample_labels.append(label[0])
    return sample_images, sample_labels

def make_stub():
    channel = implementations.insecure_channel("0.0.0.0", 7777) # update this to match your port and IP 
    stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
    return stub


def make_request(sample_images):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = "tinyimagenet"
    request.model_spec.signature_name = "serving_default"
    request.inputs['input'].CopyFrom(make_tensor_proto(np.array(sample_images), shape=[100, 64, 64, 3]))
    return request

def parse_metadata():
    """ 
    Parses the words.txt file into a map of label -> words and a list of ordered nids (index of nid = integer label).
    Also parses the val_annotations.txt file into a map of (validation_file_name --> nid)
    """
    # list all directories in the train set, the directory name is the "nid" and identifies the label
    train_dirs = py_hdfs.ls(TRAIN_DIR)
    
    # remove the path except the nid
    train_nid_list = list(map(lambda x: x.replace(TRAIN_DIR + "/", ""), train_dirs))
    
    # the number of nids equal then number of unique classes/labels
    num_classes = len(train_nid_list)
    
    # read the words.txt file that contains lines of the form "nid\twords"
    with py_hdfs.open(ID_TO_CLASS_FILE, 'r') as f:
        file_lines = f.read().decode("utf-8").split("\n")
    label_to_word = {}
    
    for l in file_lines:
        # parse each line
        wnid, word = l.split('\t')
        if wnid in train_nid_list:
            # convert the nids into integer labels by using the position in the index
            label = train_nid_list.index(wnid)
            word = str(label) + ": " + word
            # save the mapping of integer label --> words
            label_to_word[label] = word
    
    # read the val_annotations.txt file that contains lines of the form: 
    # "validation_image\tnid\tx_pos\ty_pos\tw_pos\th_pos"
    with py_hdfs.open(VAL_LABELS_FILE, 'r') as f:
        file_lines = f.read().decode("utf-8").split("\n")
    validation_file_to_nid = {}
    for l in file_lines:
        # parse each line
        tokens = l.split('\t')
        #skip corrupted lines
        if len(tokens) > 2:
            validation_img = tokens[0]
            wnid = tokens[1]
            # we only care about classification in this tutorial, not localization 
            if wnid in train_nid_list:
                validation_file_to_nid[validation_img] = wnid
    
    return train_nid_list, label_to_word, validation_file_to_nid

batch_images, batch_labels = get_one_batch()
stub = make_stub()
request = make_request(batch_images)
result = stub.Predict(request, 1000.0) #1000 secs timeout (100 examples through resnet is pretty slow on CPU)
train_nid_list, label_to_word, validation_file_to_nid = parse_metadata() 
plt.rcParams["figure.figsize"] = (14,50)
count = 0
for i in range(24):
    count += 1
    plt.subplot(12,2,count)
    plt.imshow(batch_images[i])
    plt.title('label: {}\n prediction: {}'.format(label_to_word[list(batch_labels[i]).index(1)],label_to_word[result.outputs['class_ids'].int_val[i]]))
    plt.axis("off")
plt.tight_layout()
plt.savefig("./prediction_images.png")
plt.show()
```

![prediction_images.png](./../images/prediction_images.png)
